# 1. Import dependencies

In [2]:
!pip install gymnasium[box2d] pyglet

/bin/bash: /home/irfan/miniconda3/lib/libtinfo.so.6: no version information available (required by /bin/bash)
  Using cached pyglet-2.1.11-py3-none-any.whl.metadata (7.7 kB)
  Using cached swig-4.4.0-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (3.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 20.6 MB/s  0:00:00 eta 0:00:01
Using cached swig-4.4.0-py3-none-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.9 MB)
Using cached pyglet-2.1.11-py3-none-any.whl (1.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pyglet]2m2/3 [pyglet]


In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
import os

# 2. Test Environment

** More about the environment: https://gymnasium.farama.org/environments/box2d/car_racing/

In [2]:
environment_name = 'CarRacing-v3'
env = gym.make(environment_name, render_mode='human')

/home/irfan/miniconda3/envs/RFL_Stable/lib/python3.10/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [12]:
env.reset()

(array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ...,
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ...,
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], shape=(96, 96, 3), dtype=uint8),
 {})

In [7]:
env.action_space

Box([-1.  0.  0.], 1.0, (3,), float32)

In [8]:
env.observation_space

Box(0, 255, (96, 96, 3), uint8)

In [13]:
env.render()

In [14]:
env.close()

In [ ]:
episode =5
for episode in range(episode+1):
    obs = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        score += reward
    print(f"Episode: {episode} Score: {score}")
env.close()

In [4]:
env.close()

# 3. Train Model

In [5]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [6]:
log_path = os.path.join('Training', 'Logs')
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=log_path)

Using cuda device
Wrapping the env in a VecTransposeImage.


In [7]:
model.learn(total_timesteps=100000)

Logging to Training/Logs/PPO_7
-----------------------------
| time/              |      |
|    fps             | 49   |
|    iterations      | 1    |
|    time_elapsed    | 41   |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 48           |
|    iterations           | 2            |
|    time_elapsed         | 84           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0077845077 |
|    clip_fraction        | 0.111        |
|    clip_range           | 0.2          |
|    entropy_loss         | -4.25        |
|    explained_variance   | -0.00247     |
|    learning_rate        | 0.0003       |
|    loss                 | 0.104        |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.00907     |
|    std                  | 0.996        |
|    value_loss           | 

# 4. Save Model

In [16]:
ppo_path = os.path.join('Training', 'Saved_Models', 'PPO_Driving_Model')

In [9]:
model.save(ppo_path)

In [10]:
del model

In [13]:
environment_name = 'CarRacing-v3'
env = gym.make(environment_name, render_mode='human')
env = DummyVecEnv([lambda: env])

In [17]:
import gymnasium as gym
model = PPO.load(ppo_path, env)

Wrapping the env in a VecTransposeImage.


# 5. Evaluate Model

In [18]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

/home/irfan/miniconda3/envs/RFL_Stable/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


KeyboardInterrupt: 

In [19]:
env.close()